In [1]:
import os
import random
import math

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.detection.retinanet import RetinaNet
from torchvision.models.detection.faster_rcnn import FasterRCNN

import  torchvision.transforms.functional as F

from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import xml.etree.ElementTree as ET
import collections
from torchvision.datasets.voc import VisionDataset

from functions import *
from functions_torch import *



In [2]:
params = {}
params['target_size']=(2000,1500)
params['batch_size'] = 1
params['lr'] = 0.001

voc_root = '/app/host/lacmus/dataset/full_lacmus_ds'

In [3]:
# Reworked class from pytorch (see https://pytorch.org/vision/0.8/_modules/torchvision/datasets/voc.html#VOCDetection)

class LADDDataSET(torchvision.datasets.VisionDataset):
    def __init__(
            self,
            root: str,
            image_set: str,
            transforms: Optional[Callable] = None):     
        super(LADDDataSET, self).__init__(root, transforms=transforms)
        self.image_set = image_set

        voc_root = root
        image_dir = os.path.join(voc_root, 'JPEGImages')
        annotation_dir = os.path.join(voc_root, 'Annotations')

        if not os.path.isdir(voc_root):
            raise RuntimeError('Dataset not found or corrupted.')

        splits_dir = os.path.join(voc_root, 'ImageSets/Main')
        split_f = os.path.join(splits_dir, image_set.rstrip('\n') + '.txt')

        with open(os.path.join(split_f), "r") as f:
            file_names = [x.strip() for x in f.readlines()]

        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]
        self.annotations = [os.path.join(annotation_dir, x + ".xml") for x in file_names]
        assert (len(self.images) == len(self.annotations))
        
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dictionary of the XML tree.
        """
        img = Image.open(self.images[index]).convert('RGB')
        description = LADDDataSET.parse_voc_xml(
            ET.parse(self.annotations[index]).getroot())

        # get bounding box coordinates 
        num_objs = len(description['annotation']['object'])
        boxes = []
        for l in description['annotation']['object']:
            bb = l['bndbox']
            boxes.append([int(bb['xmin']), int(bb['ymin']), int(bb['xmax']), int(bb['ymax'])])

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)         # there is only one class            
        target["labels"] = labels = torch.ones((num_objs,), dtype=torch.int64)
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    

    def __len__(self) -> int:
        return len(self.images)

    @staticmethod
    def parse_voc_xml(node: ET.Element) -> Dict[str, Any]:
        voc_dict: Dict[str, Any] = {}
        children = list(node)
        if children:
            def_dic: Dict[str, Any] = collections.defaultdict(list)
            for dc in map(LADDDataSET.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            if node.tag == 'annotation':
                def_dic['object'] = [def_dic['object']]
            voc_dict = {
                node.tag:
                    {ind: v[0] if len(v) == 1 else v
                     for ind, v in def_dic.items()}
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

In [ ]:
# Pytorch implemenation of retinanet doesn't supports train on Images without any objects (which, probably need to be fixed)
# see https://github.com/pytorch/vision/blob/master/torchvision/models/detection/retinanet.py#L475
# As a temporary solution, yet, we just filtering out empty images

splits_dir = os.path.join(voc_root, 'ImageSets/Main') 
annotation_dir = os.path.join(voc_root, 'Annotations')

with open(os.path.join(splits_dir,'train.txt'), "r") as f:
    file_names = [x.strip() for x in f.readlines()]

non_empty = []
for a in file_names:
    description = LADDDataSET.parse_voc_xml(
        ET.parse(os.path.join(annotation_dir, a + ".xml")).getroot()
    )
    num_objs = len(description['annotation']['object'])
    if num_objs > 0:
        non_empty.append(a+'\n')
        
with open(os.path.join(splits_dir,'train_non_empty.txt'), "w") as f:
    f.writelines(non_empty)

print('Total images '+str(len(file_names)), ' non empty: '+str(len(non_empty)))
                                                
                                    
        

In [ ]:
# # test DS
# im_idx = 99

# dataset = LADDDataSET('/app/host/lacmus/dataset/full_lacmus_ds','test',get_transform(train=True,target_size=params['target_size'])) 
# (image,target) = dataset[im_idx] 
# im = F.to_pil_image(image)
# draw = ImageDraw.Draw(im)

# for bb in target['boxes']:
#     draw.line([(bb[0], bb[1]), (bb[0], bb[3]), (bb[2], bb[3]),
#                (bb[2], bb[1]), (bb[0], bb[1])], width=4, fill=(255, 0, 0))

# im.show()

In [4]:
dataset_train = LADDDataSET(voc_root,'train_non_empty',get_transform(train=True,target_size=params['target_size'])) 
dataset_val = LADDDataSET(voc_root,'val',get_transform(train=False,target_size=params['target_size'])) 

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=params['batch_size'], shuffle=True, num_workers=4
     ,collate_fn=collate_fn
)

data_loader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=4
     ,collate_fn=collate_fn
)


In [ ]:
# model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True, 
#                                                            min_size=params['target_size'][0], max_size = params['target_size'][1],
#                                                            trainable_backbone_layers = 0)
# # Nees to define pretrained_backbone to use trainable_backbone_layers, otherwise it's ignored
# model.load_state_dict(torch.load('/app/host/lacmus/weights/resnet50_SDD_epoch_8.pth'), strict=False)


In [7]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True, 
                                                           min_size=params['target_size'][0], max_size = params['target_size'][1],
                                                           trainable_backbone_layers = 0)
# Nees to define pretrained_backbone to use trainable_backbone_layers, otherwise it's ignored
model.load_state_dict(torch.load('/app/host/lacmus/weights/resnet50_FRCNN_SDD_epoch_4.pth'), strict=False)



<All keys matched successfully>

In [8]:
# the computation device
device = torch.device('cuda')
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=params['lr'], momentum=0.9, weight_decay=0.0005) 
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [9]:
for epoch in range(10): # train without backbone

    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=100)
    print ("Train done, evaluating.")
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    inference_res = evaluate(model,data_loader_val)
    print('Inference done, computing mAp : ')
    print(evaluate_res(inference_res, iou_threshold = 0.5, score_threshold = 0.05))    
    print(evaluate_res(inference_res, iou_threshold = 0.6, score_threshold = 0.05))
    print('Epoch Done')
torch.save(model.state_dict(), '/app/host/lacmus/weights/resnet50_FRCNN_LADD_head.pth')

Epoch: [0]  [   0/1180]  eta: 0:21:56  lr: 0.000002  loss: 0.0995 (0.0995)  loss_classifier: 0.0232 (0.0232)  loss_box_reg: 0.0033 (0.0033)  loss_objectness: 0.0223 (0.0223)  loss_rpn_box_reg: 0.0508 (0.0508)  time: 1.1154  data: 0.7835  max mem: 1542
Epoch: [0]  [ 100/1180]  eta: 0:05:49  lr: 0.000102  loss: 0.2418 (0.3764)  loss_classifier: 0.1065 (0.1384)  loss_box_reg: 0.0901 (0.0966)  loss_objectness: 0.0510 (0.0796)  loss_rpn_box_reg: 0.0154 (0.0618)  time: 0.3444  data: 0.0626  max mem: 1624
Epoch: [0]  [ 200/1180]  eta: 0:05:14  lr: 0.000202  loss: 0.2514 (0.3737)  loss_classifier: 0.0982 (0.1256)  loss_box_reg: 0.0476 (0.1030)  loss_objectness: 0.0609 (0.0835)  loss_rpn_box_reg: 0.0296 (0.0616)  time: 0.3238  data: 0.0396  max mem: 1624
Epoch: [0]  [ 300/1180]  eta: 0:04:43  lr: 0.000302  loss: 0.2314 (0.3678)  loss_classifier: 0.0896 (0.1205)  loss_box_reg: 0.0754 (0.1145)  loss_objectness: 0.0445 (0.0733)  loss_rpn_box_reg: 0.0288 (0.0595)  time: 0.3008  data: 0.0115  max me

In [13]:
import gc
del model
del optimizer
gc.collect()
torch.cuda.empty_cache()

In [14]:
# model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True, 
#                                                            min_size=params['target_size'][0], max_size = params['target_size'][1],
#                                                            trainable_backbone_layers = 5)
# model.load_state_dict(torch.load('/app/host/lacmus/weights/resnet50_LADD_head.pth'), strict=True)


In [15]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True, 
                                                           min_size=params['target_size'][0], max_size = params['target_size'][1],
                                                           trainable_backbone_layers = 5)
model.load_state_dict(torch.load('/app/host/lacmus/weights/resnet50_FRCNN_LADD_head.pth'), strict=True)


<All keys matched successfully>

In [16]:
# the computation device
device = torch.device('cuda')
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=params['lr'], momentum=0.9, weight_decay=0.0005) 
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=4,
                                               gamma=0.1)


In [17]:
for epoch in range(10): # train with backbone now

    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=100)
    print ("Train done, evaluating.")
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    inference_res = evaluate(model,data_loader_val)
    print('Inference done, computing mAp : ')
    print(evaluate_res(inference_res, iou_threshold = 0.5, score_threshold = 0.05))    
    print(evaluate_res(inference_res, iou_threshold = 0.6, score_threshold = 0.05))
    print('Epoch Done')
    torch.save(model.state_dict(), '/app/host/lacmus/weights/resnet50_FRCNN_LADD_epoch_%i.pth'%epoch)
    

Epoch: [0]  [   0/1180]  eta: 0:30:43  lr: 0.000002  loss: 0.0811 (0.0811)  loss_classifier: 0.0214 (0.0214)  loss_box_reg: 0.0541 (0.0541)  loss_objectness: 0.0002 (0.0002)  loss_rpn_box_reg: 0.0055 (0.0055)  time: 1.5621  data: 1.0825  max mem: 5895
Epoch: [0]  [ 100/1180]  eta: 0:08:43  lr: 0.000102  loss: 0.2417 (0.2592)  loss_classifier: 0.0516 (0.0694)  loss_box_reg: 0.1218 (0.1533)  loss_objectness: 0.0124 (0.0127)  loss_rpn_box_reg: 0.0218 (0.0238)  time: 0.4739  data: 0.0095  max mem: 5895
Epoch: [0]  [ 200/1180]  eta: 0:07:54  lr: 0.000202  loss: 0.2468 (0.2546)  loss_classifier: 0.0771 (0.0690)  loss_box_reg: 0.1337 (0.1467)  loss_objectness: 0.0081 (0.0148)  loss_rpn_box_reg: 0.0138 (0.0240)  time: 0.4884  data: 0.0109  max mem: 5895
Epoch: [0]  [ 300/1180]  eta: 0:07:07  lr: 0.000302  loss: 0.2053 (0.2501)  loss_classifier: 0.0529 (0.0682)  loss_box_reg: 0.0956 (0.1420)  loss_objectness: 0.0026 (0.0151)  loss_rpn_box_reg: 0.0139 (0.0248)  time: 0.4872  data: 0.0099  max me

In [19]:
# # uncomment to test evaluation model and show detections

dataset_test = LADDDataSET(voc_root,'test',get_transform(train=False,target_size=params['target_size'])) 
data_loader_test = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=1
     ,collate_fn=collate_fn
)

image_idx = 0

cpu_device = torch.device("cpu")
model.eval()
for images, targets in data_loader_test:
    g_images = list(img.to(device) for img in images)

    if torch.cuda.is_available():
        torch.cuda.synchronize()
    outputs = model(g_images)

    outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
    res = targets, outputs
    break


im = F.to_pil_image(images[image_idx])
targets
# im = to_pil_image(dataset[10][0])
draw = ImageDraw.Draw(im)

for idx in range(len(outputs[image_idx]['boxes'])):
    width = math.ceil(outputs[image_idx]['scores'][idx]*10)
    bb = outputs[0]['boxes'][idx]
    draw.line([(bb[0], bb[1]), (bb[0], bb[3]), (bb[2], bb[3]),
               (bb[2], bb[1]), (bb[0], bb[1])], width=width, fill=(255, 0, 0))

for bb in targets[image_idx]['boxes'][:10]:
    draw.line([(bb[0], bb[1]), (bb[0], bb[3]), (bb[2], bb[3]),
               (bb[2], bb[1]), (bb[0], bb[1])], width=4, fill=(0,255, 0))
im.show()

